<img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/horizontal-primary-light.png" alt="he-black-box" width="600"/>


# Homomorphic Encryption using Duet: Data Scientist
## Tutorial 0: Basic operations



Welcome!
In this tutorial, we will show you how to use Duet with homomorphic encryption, and some use cases. This notebook shows the Data Scientist view on the operations.



Homomorphic encryption is efficient in client-server scenarios, like the one depicted in the diagram below. We will see a few use cases in which a Data Scientist can operate on encrypted data.


<img src="https://blog.openmined.org/content/images/2020/04/OM---CKKS-Graphic-v.01@2x.png" align="center" style="display: block;  margin: auto;"/>

## Setup

All modules are imported here, make sure everything is installed by running the cell below.

In [1]:
import syft as sy
import tenseal as ts
from syft.grid.client.client import connect
from syft.grid.client.grid_connection import GridHTTPConnection
from syft.core.node.domain.client import DomainClient

sy.load("tenseal")

## Connect to PyGrid

Connect to PyGrid Domain server.

In [4]:
client = connect(
    url="http://localhost:5000", # Domain Address
    credentials={"email":"user@email.com", "password":"userpwd"},
    conn_type= GridHTTPConnection, # HTTP Connection Protocol
    client_type=DomainClient) # Domain Client type

### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 0 : Now STOP and run the Data Owner notebook until Checkpoint 1.

### Check the Duet store

Now we can see all three objects and get pointers to them.


__Note__: we cannot get these without permission.

In [5]:
client.store.pandas

,ID,Tags,Description,object_type
0,<UID: b3fed70f04a6449ebb5cbead3b187a4e>,[context],,<class 'tenseal.enc_context.Context'>
1,<UID: 72327eb26d624addab7c78d120e8e588>,[enc_v1],,<class 'tenseal.tensors.ckksvector.CKKSVector'>
2,<UID: 249b8e6c602143a6a6f6d1faefa0f6d6>,[enc_v2],,<class 'tenseal.tensors.ckksvector.CKKSVector'>


### Get pointers to the encrypted vectors.

In [6]:
ctx_ptr = client.store["context"]
enc_v1_ptr = client.store["enc_v1"]
enc_v2_ptr = client.store["enc_v2"]

### Request permission to work on the encrypted vectors.

In [7]:
ctx_ptr.request(reason="I would like to get the context")
enc_v1_ptr.request(reason="I would like to get first vector")
enc_v2_ptr.request(reason="I would like to get second vector")

In [8]:
# Now we can see our requests are pending
client.requests.pandas

,Requested Object's tags,Reason,Request ID,Requested Object's ID,Requested Object's type
0,[context],I would like to get the context,<UID: d5d22bf11d474b74a4204e7990edec50>,<UID: b3fed70f04a6449ebb5cbead3b187a4e>,<class 'tenseal.enc_context.Context'>
1,[enc_v1],I would like to get first vector,<UID: 34b7d549d0b74a2fb5d8c7ce4c36dd70>,<UID: 72327eb26d624addab7c78d120e8e588>,<class 'tenseal.tensors.ckksvector.CKKSVector'>
2,[enc_v2],I would like to get second vector,<UID: b770f7925f21431b9614c3a7f2ba127e>,<UID: 249b8e6c602143a6a6f6d1faefa0f6d6>,<class 'tenseal.tensors.ckksvector.CKKSVector'>


### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 1 : Now STOP and run the Data Owner notebook until Checkpoint 2.

### Get pointers to the encrypted vectors.

In [9]:
# We can see that the requests have been answered
client.requests.pandas

""


In [10]:
ctx = ctx_ptr.get(delete_obj=False)

enc_v1 = enc_v1_ptr.get(delete_obj=False)
enc_v2 = enc_v2_ptr.get(delete_obj=False)

enc_v1.link_context(ctx)
enc_v2.link_context(ctx)

(enc_v1, enc_v2)

(<tenseal.tensors.ckksvector.CKKSVector at 0x7f02bcde8eb0>,
 <tenseal.tensors.ckksvector.CKKSVector at 0x7f02bcdf0700>)

### Compute different operations over the two vectors locally

In [11]:
result_add = enc_v1 + enc_v2
result_iadd = enc_v1 + [10, 10, 10, 10, 10]
result_sub = enc_v1 - enc_v2
result_mul = enc_v1 * enc_v2
result_pow = enc_v1 ** 3
result_neg = -enc_v1
result_poly = enc_v1.polyval([1,0,1,1]) # 1 + X^2 + X^3
result_add

### Send the result back to the Data Owner

In [12]:
result_add_ptr = result_add.send(client, pointable=True, tags=["result_add"])
result_iadd_ptr = result_iadd.send(client, pointable=True, tags=["result_iadd"])
result_sub_ptr = result_sub.send(client, pointable=True, tags=["result_sub"])
result_mul_ptr = result_mul.send(client, pointable=True, tags=["result_mul"])
result_pow_ptr = result_pow.send(client, pointable=True, tags=["result_pow"])
result_neg_ptr = result_neg.send(client, pointable=True, tags=["result_neg"])
result_poly_ptr = result_poly.send(client, pointable=True, tags=["result_poly"])


In [14]:
print(client.store.pandas)

                                        ID           Tags Description  \
0  <UID: b3fed70f04a6449ebb5cbead3b187a4e>      [context]               
1  <UID: 72327eb26d624addab7c78d120e8e588>       [enc_v1]               
2  <UID: 249b8e6c602143a6a6f6d1faefa0f6d6>       [enc_v2]               
3  <UID: df57de9cd5534e49917eb87071eb3dec>   [result_add]               
4  <UID: 9205ee2188b146c1bbc296c2aa57000b>  [result_iadd]               
5  <UID: abc8dae8c96d49aabab9f712da31f244>   [result_sub]               
6  <UID: f90223f3b8b04e15ba004a6325068cc3>   [result_mul]               
7  <UID: 7eba6e6f727f40fba1cf51a59681af28>   [result_pow]               
8  <UID: c2dab44b70d742a5ac91068c2fb3bd05>   [result_neg]               
9  <UID: 639371228f474ec0ad1b5c0d994d0536>  [result_poly]               

                                       object_type  
0            <class 'tenseal.enc_context.Context'>  
1  <class 'tenseal.tensors.ckksvector.CKKSVector'>  
2  <class 'tenseal.tensors.ckksvector

### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 2 : Now STOP and run the Data Owner notebook until the next checkpoint.

# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PySyft and TenSEAL on GitHub

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)
- [Star TenSEAL](https://github.com/OpenMined/TenSEAL)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org). #lib_tenseal and #code_tenseal are the main channels for the TenSEAL project.

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)